# HIFLD Layer Search - Proof of Concept
Simple tool to search and display HIFLD infrastructure layers

In [ ]:
# Setup
import pandas as pd
from arcgis.gis import GIS

# Load HIFLD data
df = pd.read_csv('HIFLD_Open_Crosswalk_Geoplatform.csv')
print(f"Loaded {len(df)} infrastructure layers")

# Connect to ArcGIS
gis = GIS()
print("Connected to ArcGIS")

In [ ]:
# Search function
def search_layers(keyword):
    """Search for layers by keyword"""
    keyword = keyword.lower()
    results = df[df['Layer Name'].str.lower().str.contains(keyword, na=False)]
    
    print(f"\nSearch results for '{keyword}':")
    print("=" * 50)
    
    for i, (_, row) in enumerate(results.head(10).iterrows()):
        print(f"\n{i+1}. {row['Layer Name']}")
        print(f"   Agency: {row['Agency']}")
        if pd.notna(row['Open REST Service']):
            print(f"   ✓ Map available")
    
    return results

In [ ]:
# Display layer on map
def show_on_map(layer_name):
    """Display a specific layer on the map"""
    # Find the layer
    layer = df[df['Layer Name'] == layer_name]
    
    if layer.empty:
        print(f"Layer '{layer_name}' not found")
        return None
    
    row = layer.iloc[0]
    url = row['Open REST Service']
    
    if pd.isna(url):
        print(f"No map service available for {layer_name}")
        return None
    
    # Create map
    print(f"\nDisplaying: {layer_name}")
    print(f"Agency: {row['Agency']}")
    print(f"URL: {url}")
    
    map_widget = gis.map('USA')
    
    try:
        # Handle MapServer sublayer URLs (e.g., ending with /51)
        if '/MapServer/' in url and url[-1].isdigit():
            # Extract base URL and layer ID
            base_url = url.split('/MapServer/')[0] + '/MapServer'
            layer_id = int(url.split('/')[-1])
            print(f"Adding MapServer sublayer {layer_id} from {base_url}")
            map_widget.add_layer({'url': base_url, 'sublayers': [layer_id]})
        else:
            # Regular feature service
            map_widget.add_layer({'url': url})
        
        print("✓ Layer added successfully!")
    except Exception as e:
        print(f"Error adding layer: {e}")
        # Try alternative method
        try:
            print("Trying alternative method...")
            map_widget.add_layer({'url': url})
            print("✓ Layer added with alternative method!")
        except Exception as e2:
            print(f"Both methods failed: {e2}")
    
    return map_widget

## Try it out!
Run the cells below to search for different infrastructure types

In [ ]:
# Search for fire stations
fire_results = search_layers('fire')

In [ ]:
# Display fire stations on map
fire_map = show_on_map('Fire and Emergency Medical Service (EMS) Stations')
fire_map

In [ ]:
# Search for hospitals
hospital_results = search_layers('hospital')

In [ ]:
# Search for schools
school_results = search_layers('school')

In [ ]:
# Combined search and map
def quick_map(keyword):
    """Search and display first result"""
    results = search_layers(keyword)
    
    # Find first layer with map
    for _, row in results.iterrows():
        if pd.notna(row['Open REST Service']):
            return show_on_map(row['Layer Name'])
    
    print("No mappable layers found")
    return None

In [ ]:
# Quick map example
power_map = quick_map('power')
power_map

## Save Map Example
How to save a map with metadata

In [ ]:
# Save map function
def save_map_suggestion(layer_name):
    """Generate save suggestions for a layer"""
    layer = df[df['Layer Name'] == layer_name].iloc[0]
    
    # Suggest metadata
    title = f"HIFLD {layer_name} Map"
    summary = f"Map showing {layer_name} from {layer['Agency']}"
    tags = ['HIFLD', 'infrastructure', layer['Agency'], layer_name.split()[0]]
    
    print("Suggested map metadata:")
    print(f"\nTitle: {title}")
    print(f"Summary: {summary}")
    print(f"Tags: {', '.join(tags)}")
    
    return {'title': title, 'summary': summary, 'tags': tags}

# Example
save_map_suggestion('Colleges and Universities')